In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import re
import operator
import ast
import numpy as np 
import pandas as pd
import warnings
import ast
import math
import matplotlib.pyplot as plt
import matplotlib.style as style
import tensorflow as tf
from PIL import Image, ImageDraw 
from tqdm import tqdm
from dask import bag
from glob import glob
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet,MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
import numpy as np
from keras.models import model_from_yaml


class VotingModel(object):

    def __init__(self, model_list, voting='hard',
                 weights=None, nb_classes=None):
     
        self.model_list = model_list
        self.voting = voting
        self.weights = weights
        self.nb_classes = nb_classes

        if voting not in ['hard', 'soft']:
            raise 'Voting has to be either hard or soft'

        if weights is not None:
            if len(weights) != len(model_list):
                raise ('Number of models {0} and length of weight vector {1} has to match.'
                       .format(len(weights), len(model_list)))

    def predict(self, X, batch_size=128, verbose=0):
        predictions = list(map(lambda model: model.predict(X, batch_size, verbose), self.model_list))
        nb_preds = len(X)

        if self.voting == 'hard':
            for i, pred in enumerate(predictions):
                pred = list(map(
                    lambda probas: np.argmax(probas, axis=-1), pred
                ))
                predictions[i] = np.asarray(pred).reshape(nb_preds, 1)
            argmax_list = list(np.concatenate(predictions, axis=1))
            votes = np.asarray(list(
                map(lambda arr: max(set(arr)), argmax_list)
            ))
        if self.voting == 'soft':
            for i, pred in enumerate(predictions):
                pred = list(map(lambda probas: probas * self.weights[i], pred))
                predictions[i] = np.asarray(pred).reshape(nb_preds, self.nb_classes, 1)
            weighted_preds = np.concatenate(predictions, axis=2)
            weighted_avg = np.mean(weighted_preds, axis=2)
            votes = np.argmax(weighted_avg, axis=1)

        return votes


def voting_model_from_yaml(yaml_list, voting='hard', weights=None):
    model_list = map(lambda yml: model_from_yaml(yml), yaml_list)
    return VotingModel(model_list, voting, weights)

Using TensorFlow backend.


In [6]:
model1 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/classifier_for_ensemble/Benny1.model')
model1.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ["accuracy"])
#model1.summary()
model2 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/classifier_for_ensemble/Benny2.model')
model2.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ["accuracy"])
#model2.summary()
model3 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/classifier_for_ensemble/Benny3.model')
model3.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ["accuracy"])
#model3.summary()
model4 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/classifier_for_ensemble/Benny4.model')
model4.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ["accuracy"])
#model4.summary()
model5 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/classifier_for_ensemble/19.model')
model5.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ["accuracy"])
#model5.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# Majority Rule (hard) Voting

mv_clf = VotingModel(model_list=[model1, model2, model3, model4, model5]) # 整合5個模型

clf_labels = ['model1', 'model2', 'model3', 'model4', 'model5', 'Majority voting']
all_clf = [model1, model2, model3, model4, model5, mv_clf]

In [8]:
class_paths =glob('/content/drive/My Drive/Colab Notebooks/input/train/*.csv')
numstonames = {i: v[14:-4].replace(" ", "_") for i, v in enumerate(class_paths)}
for i in numstonames:
  numstonames[i]= numstonames[i].replace("/My_Drive/Colab_Notebooks/input/train/","")
print(numstonames)

namestonums = {v:k for k,v in numstonames.items()}

num_classes = 30
imheight, imwidth = 64, 64  
ims_per_class = 2000

def draw_it(strokes):
    image = Image.new("P", (64,64), color=255)
    image_draw = ImageDraw.Draw(image)
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=0, width=5)
    image = image.resize((imheight, imwidth))
    return np.array(image)/255.


{}


In [9]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/input/test.csv')
imagebag = bag.from_sequence(test.drawing.values).map(draw_it)
testarray = np.array(imagebag.compute())
testarray = np.reshape(testarray, (testarray.shape[0], imheight, imwidth, 1))

X_test = testarray
y_test= [namestonums[w.replace(" ", "_") ] for w in test.word]


y_test = keras.utils.to_categorical(y_test, num_classes)
X_test = X_test.reshape(X_test.shape[0], imheight, imwidth, 1)

FileNotFoundError: ignored

In [0]:
del test['Unnamed: 0']

test

,drawing,word
0,"[[[51, 43, 17, 2, 0, 4, 21, 28, 60, 74, 79, 79...",key
1,"[[[115, 118, 125, 134, 154, 164, 174, 173, 157...",swan
2,"[[[67, 59, 29, 7, 0, 2, 16, 40, 59, 87, 121, 1...",light bulb
3,"[[[11, 8, 17, 48], [39, 79, 129, 128]], [[23, ...",bed
4,"[[[0, 21, 46, 64, 83, 118, 139, 156, 221, 245,...",roller coaster
5,"[[[8, 6, 11, 7, 0, 5, 80, 134, 181, 248, 226, ...",door
6,"[[[95, 74, 60, 34, 4, 0, 9, 23, 51, 64, 89, 10...",key
7,"[[[7, 8, 5, 1, 11, 32, 76, 121, 234, 254, 253,...",The Great Wall of China
8,"[[[62, 63, 88, 89, 97, 121, 125, 127, 127, 112...",paintbrush
9,"[[[97, 60, 36, 30, 14, 1, 1, 15, 45, 49, 52], ...",light bulb


In [0]:
y_test_pred = mv_clf.predict(X_test)

In [0]:
predict_label = [numstonames[y_test_pred[i]] for i in range(0,41)]

test['word'] = predict_label
test.to_csv('/content/drive/My Drive/Colab Notebooks/test_predict.csv')